# Wikipedia page history evaluation

Wikipedia pages can be vandalized. Wikipedia itself prioritizes having low amount of false positives from automatic checkers, which can cause the latest version to be vandalized. Try to find a stable – but recent –  version of the page to be used as a context.


## Approach testing – Editor history

Use editor edit-history to see if the user can be considered a "trustworthy".

Recent edits are collected from page, and every editor is checked how their recent edits have been reverted.

TODO: implement checks that allows self-reverts.
TODO: Check that revert is not reverted back.

In [88]:
!pip install -q requests ratelimit
import requests
import ratelimit
try:
    import requests_cache
    requests_cache.install_cache("/tmp/wp-api-cache")
except ImportError:
    print("No requests cache available")
    pass

# Disable hugginface stats
import os
os.environ['HF_HUB_DISABLE_TELEMETRY'] = "1"

import logging
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

# revision tags that are used to indicate revision is reverted by later edit.
# https://en.wikipedia.org/wiki/Special:Tags
MW_REVERTED_TAGS: set[str] = {"mw-reverted"}

_session = requests.Session()

@ratelimit.sleep_and_retry
@ratelimit.limits(calls=5, period=1)
def wpapi(params, headers={}, site="en.wikipedia.org"):
    url = f"https://{site}/w/api.php"
    params.setdefault("format", "json")
    response = _session.get(url, params=params, headers=headers)
    response.raise_for_status()

    data = response.json()
    if "error" in data:
        raise Exception(f"WPAPI Error: {data["error"]}")

    return data




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Detecting revert

Wikipedia doesn't contain reliable labeled information that can be used to indicate if the edit is a reverted. Sometimes the edit might contain tags as indicator, sometimes not. We can use the edit comment to check if the edit is a reverting previous work. If the comment contains keywords such as "revert", "undid", or "rv" plus the user's name or revision number, we can consider the edit as a revert.

TODO: Check if the comment mentions that it's reverting to an earlier commit.

In [89]:
# Function to detect if the edit is a revert

import re
from typing import TypedDict
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

revert_phrases = [
    "Reverted edits by {user!r}",
    "Reverted 1 pending edit by [[Special:Contributions/{user}|{user}]] to revision <number> by <username>: <reason description>",
    "Reverting possible vandalism by {user!r} to version by <username>",  # Cluebot NG
    "Reverted edit by [[Special:Contribs/{user}|{user}]] ([[User talk:{user}|talk]]) to last version by <username>",
    "Undid revision [[Special:Diff/{revid}|{revid}]] by [[Special:Contributions/{user}|{user}]] ([[User talk:{user}|talk]]): <reason description>",
    # "Reverted edit ",
    # "Undid revision",
    # "Restored revision <number> by <username>",
    # "Revert",
    # "Rollback",
    # "Undo",
    # "Reverted"
]

class PageRevision(TypedDict, total=False):
    pageid: int
    revid: int
    parentid: int
    user: str
    timestamp: str
    comment: str
    tags: list[str]


def is_revert(revision: PageRevision, original_edit, threshold=0.75):
    """
    Indicate if the edit is a revert.

    Uses sentence-transformers to semantically compare the comment with
    predefined revert indicator phrases.

    Returns True if the comment is likely a revert.
    """

    log = logger.getChild("is_revert")
    log.debug("Checking if revision %r is a revert", revision)

    comment = revision.get("comment", "")
    # Remove comment (/* ... */) from the comment that is used to indicate section
    comment = re.sub(r'/\* .*? \*/', '', comment)

    if not comment:
        log.debug("Revision %r has no comment, skipping", revision)
        return False

    # If the revision ID or username is not mentioned, it's not likely a revert. 
    username = original_edit.get("user", "")
    revision_id_str = str(original_edit.get("revid"))

    rev_pattern = r'\b' + re.escape(revision_id_str) + r'\b'
    username_pattern = r'\b' + re.escape(username) + r'\b'
    
    has_revision_id = bool(re.search(rev_pattern, comment))
    user_mentioned = bool(re.search(username_pattern, comment))

    if not any((has_revision_id, user_mentioned)):
        log.debug("Revision ID %s or username %r not mentioned in comment: %r", revision_id_str,  username, comment)
        return False

    formatted_revert_phrases = [s.format(**original_edit) for s in revert_phrases]

    # Encode the revert phrases.
    revert_embeddings = model.encode(formatted_revert_phrases, convert_to_tensor=True, show_progress_bar=False)

    # Encode the comment.
    comment_embedding = model.encode(comment, convert_to_tensor=True, show_progress_bar=False)
    # Compute (cosine) similarity with the pre-encoded revert phrases.
    cosine_scores = model.similarity(comment_embedding, revert_embeddings)

    # Use the maximum similarity score as the indicator.
    # Using for-loop to be able to log the similarity score for each pattern.
    #max_score = cosine_scores.max().item()
    max_score = 0.0
    for i, score in enumerate(cosine_scores[0]):
        max_score = max(score, max_score)
        log.debug(f" {max_score:.2f} - Pattern {formatted_revert_phrases[i]!r} match to {comment!r}")

    # Set a threshold for similarity (this needs to be fine-tuned).
    if max_score >= threshold:
        log.info("Revision %r is a revert", revision)
        return True
    return False


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [90]:
def fetch_last_revisions(page_title, limit=20) -> list[PageRevision]:
    """
    Fetches the last `limit` revisions for the given Wikipedia page title.
    """
    params = {
        'action': 'query',
        'prop': 'revisions',
        'titles': page_title,
        'rvlimit': limit,
        'rvprop': 'ids|timestamp|user|comment|tags',
    }

    data = wpapi(params)
    pages = data.get('query', {}).get('pages', {})
    revisions = []
    # The query returns a dictionary keyed by pageid
    for page_id, page in pages.items():
        if "missing" in page:
            print(f"The page '{page_title}' does not exist on Wikipedia.")
            return None
        revisions = page.get('revisions', [])
        revisions.extend([PageRevision(**rev, pageid=page_id) for rev in revisions])
    return revisions

page_edits = fetch_last_revisions("Sofi_Tukker", limit=15)
display(page_edits[-3:])



[{'revid': 1254954349,
  'parentid': 1254925063,
  'user': 'Binksternet',
  'timestamp': '2024-11-02T13:09:40Z',
  'comment': 'Reverted 1 edit by [[Special:Contributions/31.217.4.176|31.217.4.176]] ([[User talk:31.217.4.176|talk]]) to last revision by 162 etc.',
  'tags': ['mw-undo', 'twinkle', 'mw-reverted'],
  'pageid': '49079438'},
 {'revid': 1254925063,
  'parentid': 1254811923,
  'user': '31.217.4.176',
  'anon': '',
  'timestamp': '2024-11-02T10:02:53Z',
  'comment': '',
  'tags': ['mobile edit',
   'mobile web edit',
   'visualeditor',
   'mw-reverted',
   'disambiguator-link-added'],
  'pageid': '49079438'},
 {'revid': 1254811923,
  'parentid': 1254792543,
  'user': '162 etc.',
  'timestamp': '2024-11-01T20:19:03Z',
  'comment': 'Undid revision [[Special:Diff/1254792543|1254792543]] by [[Special:Contributions/31.217.4.176|31.217.4.176]] ([[User talk:31.217.4.176|talk]])',
  'tags': ['mw-undo', 'wikieditor'],
  'pageid': '49079438'}]

In [91]:
def fetch_user_contributions(username: str, limit=100) -> list[PageRevision]:
    """
    Fetches up to `limit` contributions made by the given user.
    """
    params = {
        "action": "query",
        "list": "usercontribs",
        "ucuser": username,
        "uclimit": limit,
        "ucprop": "ids|title|timestamp|comment|tags",
        "format": "json"
    }
    data = wpapi(params=params)

    # Only consider the page edits
    return [PageRevision(edit) for edit in data.get("query", {}).get("usercontribs", []) if edit.get("ns", 0) == 0]

username = page_edits[-1]['user']
user_contributions = fetch_user_contributions(username, limit=3)
print(f"Last 3 contributions by {username!r}")
display(user_contributions)

Last 3 contributions by '162 etc.'


[{'userid': 41625025,
  'user': '162 etc.',
  'pageid': 72768531,
  'revid': 1276686643,
  'parentid': 1276503368,
  'ns': 0,
  'title': '2025 Formula One World Championship',
  'timestamp': '2025-02-20T06:54:59Z',
  'comment': '/* Entries */ fix links',
  'tags': ['mw-reverted', 'wikieditor']},
 {'userid': 41625025,
  'user': '162 etc.',
  'pageid': 72504121,
  'revid': 1276685886,
  'parentid': 1276148117,
  'ns': 0,
  'title': '2024 Formula One World Championship',
  'timestamp': '2025-02-20T06:48:56Z',
  'comment': "/* World Constructors' Championship standings */ links to proper targets",
  'tags': ['mw-reverted', 'wikieditor']},
 {'userid': 41625025,
  'user': '162 etc.',
  'pageid': 21836491,
  'revid': 1276683139,
  'parentid': 1186589895,
  'ns': 0,
  'title': 'TMO',
  'timestamp': '2025-02-20T06:26:21Z',
  'comment': '',
  'tags': ['wikieditor']}]

In [92]:

from datetime import datetime, timedelta
from typing import List

def get_next_revisions(revision: PageRevision, n) -> list[PageRevision]:
    """
    Fetches the next `n` revisions after the given timestamp on the specified page.
    """

    revision_timestamp = revision['timestamp']

    dt = datetime.strptime(revision_timestamp, "%Y-%m-%dT%H:%M:%SZ")
    dt_next = dt + timedelta(seconds=1)
    start_timestamp = dt_next.strftime("%Y-%m-%dT%H:%M:%SZ")

    params = {
        "action": "query",
        "prop": "revisions",
        "pageids": revision['pageid'],
        "rvstart": start_timestamp,
        "rvdir": "newer",
        "rvlimit": n,
        "rvprop": "ids|timestamp|comment|tags"
    }
    data = wpapi(params)
    logger.debug("Next revisions: %r", data)
    pages = data.get("query", {}).get("pages", {})
    revisions_list = []
    for pid, revision in pages.items():
        revisions = revision.get("revisions", [])
        revisions_list.extend([PageRevision(**rev, pageid=pid) for rev in revisions])
    return revisions_list


def check_contribution_reverted(revision: PageRevision, num_later_edits=10) -> List[bool | PageRevision]:
    """
    Has the contribution been reverted?

    Checks if a particular contribution was reverted by examining the next
    `num_later_edits` revisions on the page. If any of those revisions' comments
    indicate a revert targeting the given username or revision, returns True.

    Returns list of N>0 if the edit is reverted.
    """

    # Check if the edit has the "mw-reverted" tag.
    cx_tags = MW_REVERTED_TAGS & set(revision.get("tags", []))
    logger.debug("Revision tags: %r", revision.get("tags", []))
    if len(cx_tags):
        logger.info("[REVERTED] Revision %d is tagged as reverted with tag(s) %r", revision['revid'], cx_tags)
        return [True]

    # Check the follow-up revisions if they mention this edit.
    next_revs = get_next_revisions(revision, num_later_edits)
    logger.debug("Number of following revisions: %d", len(next_revs))

    r = []
    for followup_revision in next_revs:
        comment = followup_revision.get("comment", "")

        logger.debug(f"Checking revision {followup_revision['revid']} with comment: {comment!r}")

        if is_revert(followup_revision, revision):
            # TODO: We should recurse to see, if reverting revision has been reverted
            logger.info("[REVERTED] Comment %r is reverted by %r (revid:%d)", revision['comment'], comment, followup_revision['revid'])
            r.append(followup_revision)
    return r

earliest_contrib = page_edits[-1]
print("Checking edit:", earliest_contrib)

reverts = list(check_contribution_reverted(earliest_contrib))

display(reverts)


Checking edit: {'revid': 1254811923, 'parentid': 1254792543, 'user': '162 etc.', 'timestamp': '2024-11-01T20:19:03Z', 'comment': 'Undid revision [[Special:Diff/1254792543|1254792543]] by [[Special:Contributions/31.217.4.176|31.217.4.176]] ([[User talk:31.217.4.176|talk]])', 'tags': ['mw-undo', 'wikieditor'], 'pageid': '49079438'}


[]

In [51]:
# Final run - Collect edits, collect editors from the edits, and check how many of the lastest edits by the editor is reverted.

from typing import NamedTuple


class RevertStats(NamedTuple):
    rate: float
    total: int
    reverted: int

def compute_user_revert_rate(username: str, contrib_limit: int = 100, later_edits_to_check: int = 10) -> RevertStats:
    """
    Computes the revert rate for a user by checking up to `contrib_limit`
    of their contributions. For each contribution, it checks the next
    `later_edits_to_check` revisions to see if it was reverted.
    
    Returns a tuple: (revert_rate, total_checked, total_reverted).
    """
    contributions = fetch_user_contributions(username, limit=contrib_limit)
    if not contributions:
        return RevertStats(None, 0, 0)
    total = 0
    reverted = 0
    for contrib in contributions:
        total += 1
        if check_contribution_reverted(contrib, num_later_edits=later_edits_to_check):
            reverted += 1
        # time.sleep(0.2)
    rate = (reverted / total) if total > 0 else 0
    return RevertStats(rate, total, reverted)

username = "2A05:4F44:1701:2500:D010:2C76:10A4:52C7"
rate, total, reverted = compute_user_revert_rate(username, 10)
print(f"Revert rate for {username}: {rate:.2%} ({reverted}/{total})")


INFO:__main__:[REVERTED] Revision 1256966548 is tagged as reverted with tag(s) {'mw-reverted'}
INFO:__main__:[REVERTED] Revision 1256962846 is tagged as reverted with tag(s) {'mw-reverted'}
INFO:__main__:[REVERTED] Revision 1256962795 is tagged as reverted with tag(s) {'mw-reverted'}


Revert rate for 2A05:4F44:1701:2500:D010:2C76:10A4:52C7: 100.00% (3/3)


In [54]:
# Rank the editors by revert rate

editors = set([edit['user'] for edit in page_edits])

revert_rates = {editor: compute_user_revert_rate(editor) for editor in editors}

sorted_revert_rates = sorted(revert_rates.items(), key=lambda x: x[1].rate, reverse=True)

for editor, stats in sorted_revert_rates:
    print(f"{editor}: {stats.rate:.2%} ({stats.reverted}/{stats.total})")


INFO:__main__:[REVERTED] Revision 1276409764 is tagged as reverted with tag(s) {'mw-reverted'}
INFO:__main__:[REVERTED] Revision 1276401089 is tagged as reverted with tag(s) {'mw-reverted'}
INFO:__main__:[REVERTED] Revision 1276400649 is tagged as reverted with tag(s) {'mw-reverted'}
INFO:__main__.is_revert:Revision {'revid': 1276308341, 'parentid': 1276307449, 'timestamp': '2025-02-18T02:39:27Z', 'comment': 'Restored revision 1276239677 by [[Special:Contributions/Duckmather|Duckmather]] ([[User talk:Duckmather|talk]])', 'tags': ['mw-undo', 'twinkle'], 'pageid': '79642'} is a revert
INFO:__main__:[REVERTED] Comment 'Undid revision [[Special:Diff/1276211099|1276211099]] by [[Special:Contributions/Elena Ene D-Vasilescu|Elena Ene D-Vasilescu]] ([[User talk:Elena Ene D-Vasilescu|talk]]): malformed promotional spam' is reverted by 'Restored revision 1276239677 by [[Special:Contributions/Duckmather|Duckmather]] ([[User talk:Duckmather|talk]])' (revid:1276308341)
INFO:__main__:[REVERTED] Revi

31.217.4.176: 100.00% (9/9)
2A05:4F44:1701:2500:D010:2C76:10A4:52C7: 100.00% (3/3)
Chetsford: 13.04% (3/23)
GreenC bot: 5.10% (5/98)
Binksternet: 4.48% (3/67)
Duckmather: 1.41% (1/71)
162 etc.: 0.00% (0/76)
MusikBot II: 0.00% (0/82)
107.116.79.140: 0.00% (0/3)
Arjayay: 0.00% (0/95)
174.92.221.85: 0.00% (0/5)


### Group based power-level

Wikipedia uses RBAC (Role Based Access Control) to determine the rights of the user. In theory, the rights should have been granted by their merit. Use the groups as indicator of "power-level" of the user. More powerful the user is, the more likely they are to be trusted – according to wikipedia community.



In [59]:
# Map of groups to an power level.

from collections import OrderedDict
import ipaddress
from typing import Iterable

# Group to use if the IP address is blocked.
USER_BLOCKED_GROUP = "blocked"

GROUP_TRUST_LEVELS = {
    "banned": -10000,
    "blocked": -10000,
    "bot": 10,
    "autoconfirmed": 1,
    "extendedconfirmed": 20,
    "rollback": 50,
    "sysop": 100,
    "bureaucrat": 400,
    "checkuser": 350,
    "oversight": 350,
    "steward": 350,
}

def check_ip_blocked(ip: str) -> bool:
    """
    Check if the IP address is blocked on Wikipedia.
    """
    params = {
        "action": "query",
        "list": "blocks",
        "bkip": ip,
        "format": "json"
    }
    data = wpapi(params)
    block = data.get("query", {}).get("blocks", [])
    if bool(block):
        logger.info("IP %r is blocked: %r", ip, block[0].get("reason", ""))
        return True
    return False


def get_power_levels(users: Iterable[str]) -> List[int]:
    """
    Get the power levels of the given users.

    The power level is a sum of the trust levels of the groups the user is in. Higher
    power level indicates more trust. The trust levels are defined in the GROUP_TRUST_LEVELS
    dictionary.

    :param users: List of usernames or IP addresses.
    :return: List of power levels for each user.

    """

    # Initialize the user map with 0 trust level.
    user_map = OrderedDict((u, 0) for u in users)

    # Separate IP addresses from usernames, they need to be queried differently.
    ips, usernames = [], []
    for entry in users:
        try:
            if ipaddress.ip_address(entry):
                ips.append(str(entry))
        except ValueError:
            usernames.append(entry)

    # Query the user groups and blockinfo for the given usernames.
    params = {
        "action": "query",
        "list": "users",
        "ususers": "|".join(usernames),
        "usprop": "groups|blockinfo",
    }
    data = wpapi(params)

    for user_info in data.get("query", {}).get("users", []):
        # If user is tagged as "invalid"
        if 'invalid' in user_info:
            logger.warning("User %r is invalid: %r", user_info.get('name', ""), user_info.get('invalidreason', ""))
            continue

        total_trust = 0
        user = user_info['name']
        groups = user_info.get("groups", [])

        if user_info.get("blockid", None) is not None:
            logger.info("User %r is blocked: %r", user, user_info.get('blockreason', ""))
            groups.append(USER_BLOCKED_GROUP)

        for group in set(groups):
            group_key = group.lower()  # Ensure case-insensitive matching.
            trust = GROUP_TRUST_LEVELS.get(group_key, 0)  # Default to 0 if not mapped.
            total_trust += trust
        user_map[user] = total_trust

    # Check if the IP address is blocked.
    for ip in ips:
        if check_ip_blocked(ip):
            user_map[ip] += GROUP_TRUST_LEVELS[USER_BLOCKED_GROUP]

    return list(user_map.values())

def rank_by_power_level(users: Iterable[str]) -> List[str]:
    """
    Rank the users by their power levels.

    :param users: List of usernames or IP addresses.
    :return: List of usernames sorted by power level.
    """
    power_levels = get_power_levels(users)
    return [u for p, u in sorted(zip(power_levels, users), reverse=True) if p > 0]

level_check_users = list(editors) + ["Jayson Cesar Bautista"]

power_levels = get_power_levels(level_check_users)
for editor, power in zip(level_check_users, power_levels):
    print(f"{editor}: {power}")


INFO:__main__:User 'Jayson Cesar Bautista' is blocked: '[[WP:SOCK|Sock puppetry]]'
INFO:__main__:IP '2A05:4F44:1701:2500:D010:2C76:10A4:52C7' is blocked: '[[WP:Disruptive editing|Disruptive editing]], block evasion, vandalism, see 2A05:4F44:1704:D500:0:0:0:0/64'


162 etc.: 21
Binksternet: 21
MusikBot II: 111
107.116.79.140: 0
Arjayay: 21
Duckmather: 21
31.217.4.176: 0
Chetsford: 101
174.92.221.85: 0
GreenC bot: 11
2A05:4F44:1701:2500:D010:2C76:10A4:52C7: -10000
Jayson Cesar Bautista: -10000


INFO:__main__:User 'Jayson Cesar Bautista' is blocked: '[[WP:SOCK|Sock puppetry]]'
INFO:__main__:IP '2A05:4F44:1701:2500:D010:2C76:10A4:52C7' is blocked: '[[WP:Disruptive editing|Disruptive editing]], block evasion, vandalism, see 2A05:4F44:1704:D500:0:0:0:0/64'


MusikBot II
Chetsford
Duckmather
Binksternet
Arjayay
162 etc.
GreenC bot


### Review log

Rank the revisions based on their review status. Some wikipedias employ review flags to indicate that the revision has been reviewed by a trusted user. Example from Finnish wikipedia: https://fi.wikipedia.org/wiki/Ohje:Sivujen_arviointi

Having a review flag is NOT a guarantee that the information is correct, but it can be used as an indicator that the revision has no obvious vandalism.

Revisions with flags are preferred, and reviews from other users are ranked above having a "trusted" status. 


In [116]:
REVIEWED_TAGS = {"stable", "reviewed"}

def get_page_revisions_with_review(page_title, rvlimit=20):
    """
    Retrieves the latest revisions for a page along with flagged (review) info.
   
    Revisions with "stable" are ranked above non-reviewed revisions, and revisions made by other users are ranked above
    the user's own revisions.
    """
    params = {
        # "action": "query",
        # "titles": page_title,
        # "prop": "revisions|flagged",
        # "rvlimit": rvlimit,
        # "rvprop": "ids|timestamp|user|comment|tags|flags",
        # "format": "json"
        "action": "query",
        "prop": "revisions",
        "titles": page_title,
        "rvlimit": rvlimit,
        "rvdir": "older",
        "rvprop": "ids|timestamp|user|flagged",
        "format": "json"
    }
    data = wpapi(params, site="fi.wikipedia.org")

    pages = data.get("query", {}).get("pages", {})
    r = []
    for page_id, page in pages.items():
        revisions = page.get("revisions", [])
        
        for rev in revisions:
            score = 0
            flagged = rev.get("flagged", {})
            if not flagged: continue
            if flagged.get("level_text", "") in REVIEWED_TAGS:
                score += 1
                if rev["user"] != flagged.get("user", None):
                    # If the revision is reviewed and the reviewer is different from the editor, give extra points.
                    score += 1

            r.append((score, PageRevision(rev)))
    # Sort by score in descending order.
    r.sort(key=lambda x: x[0], reverse=True)
    return [rev for score, rev in r]

page_title = "Niklas_Anttila"
page_revisions = get_page_revisions_with_review(page_title)
display(page_revisions)


[{'revid': 22638561,
  'parentid': 22595899,
  'user': '87.95.32.17',
  'anon': '',
  'timestamp': '2024-09-16T19:46:57Z',
  'flagged': {'user': 'BladeJ',
   'timestamp': '2024-10-06T19:56:57Z',
   'level': 0,
   'level_text': 'stable',
   'tags': {'accuracy': 1}}},
 {'revid': 22595899,
  'parentid': 22595855,
  'user': 'Kuosmanono',
  'timestamp': '2024-08-25T22:41:38Z',
  'flagged': {'user': 'BladeJ',
   'timestamp': '2024-09-07T13:21:10Z',
   'level': 0,
   'level_text': 'stable',
   'tags': {'accuracy': 1}}},
 {'revid': 22358375,
  'parentid': 22273183,
  'user': 'SaMSUoM',
  'timestamp': '2024-04-29T16:50:05Z',
  'flagged': {'user': 'Seegge',
   'timestamp': '2024-07-28T20:44:51Z',
   'level': 0,
   'level_text': 'stable',
   'tags': {'accuracy': 1}}},
 {'revid': 22079023,
  'parentid': 22037750,
  'user': '176.72.103.70',
  'anon': '',
  'timestamp': '2023-12-30T10:26:47Z',
  'flagged': {'user': 'Elastul',
   'timestamp': '2024-01-01T08:22:44Z',
   'level': 0,
   'level_text': 's